# Analysis of chemo diversity on pharmacological and biological activities

The [dataset](results/activities_2022-01-29_16-33-05.csv) is obtained from the Scopus downloader version 2.0 from the [base set](data/activities.csv) of chemical compounds and activities.

**TODO** check:

- <https://stackoverflow.com/questions/53927460/select-rows-in-pandas-multiindex-dataframe>
- <https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#multiindex-query-syntax>


## Loading data from Scopus

### Boilerplate

Some Python configuration.


In [1]:
from pathlib import Path
from itertools import islice, product
from pprint import pprint

import pandas as pd
import numpy as np
from scipy import linalg

import prince
# BUG !

import seaborn as sns
import matplotlib.pyplot as plt
import holoviews as hv

import biblio_extractor as bex

np.set_printoptions(precision=4, suppress=True)
pd.set_option("display.max_columns", 10)
pd.set_option("display.max_rows", 20)
pd.set_option("display.min_rows", 10)
# pd.set_option('display.expand_frame_repr', False)

DATASET_FILENAME = Path("results/activities_2022-01-29_16-33-05.csv")


ImportError: cannot import name 'url' from 'django.conf.urls' (/home/romulus/.local/lib/python3.10/site-packages/django/conf/urls/__init__.py)

# Loading the CSV file

Now, we load the dataset we'll use in this Notebook. It is generated using [our tool](biblio_extractor.py) in the same repository.
The dataset is a 106 \* 66 matrix, with rows and columns indexed by three levels (using Pandas's `MultiIndex`) as follows:

- level 0 is _class_ (chemical, biological or pharmacological class)
- level 1 is _name_ (compound or activity, a.k.a. keyword)
- level 2 is _kind_ (either _w/o_ for _without_ or _w/_ for _with_)

The matrix is indexed by _two_ disjoint finite sets of keywords:

- **rows**: a set of 53 (chemical) coumpounds = {acridine, triterpene, ...}
- **columns**: a set of 33 (biological, pharmacological) activities = {germination, cytotoxicity, ...}

An example is shown below, restricted to two compounds and two activities.
Note that this is **not** an extract of the whole dataset, the (w/o, w/o) cells being smaller here.

|                   |            |     | germination | germination | cytotoxicity | cytotoxicity |
| ----------------- | ---------- | --- | ----------- | ----------- | ------------ | ------------ |
|                   |            |     | allelopathy | allelopathy | pharmaco     | pharmaco     |
|                   |            |     | w/o         | w/          | w/o          | w/           |
| alkaloid          | acridine   | w/o | 2100        | 32          | 28           | 2104         |
| alkaloid          | acridine   | w/  | 1294        | 11          | 9            | 1296         |
| terpenoid/terpene | triterpene | w/o | 1283        | 11          | 9            | 1285         |
| terpenoid/terpene | triterpene | w/  | 2111        | 32          | 28           | 2115         |

Let $M$ be this matrix, and for each couple of keywords made of a compound and and activity, call $M_{ij} = (c_i, a_j)$, the **ij confusion submatrix**.
Assume that $M_ij$ is of the form :
\begin{bmatrix}
U & V\\
X & Y
\end{bmatrix}

Where :

- $U = (\text{w/o}, \text{w/o})$ is the number of papers that have **neither** the $c_i$ compound **nor** the $a_j$ activity;
- $V = (\text{w/o}, \text{w/})$ is the number of papers that have the $a_j$ activity but **not** the $c_i$ compound;
- $X = (\text{w/}, \text{w/o})$ is the number of papers that have the $c_i$ compound but **not** the $a_j$ activity;
- $Y = (\text{w/}, \text{w/})$ is the number of papers that have **both** the $c_i$ compound **and** the $a_j$ activity.

We avoid the open world hypothesis by restricting the analysis to the paper in the domain $D$,
which is the set of papers that have at least one compound and one activity.
By construction:

- $U + V$ and $X + Y$ are constants for each $c_i$ (whatever the choice of $a_j$) and is the total number of papers in $D$ with the $c_i$ comppound;
- $U + X$ and $V + Y$ are constants for each $a_j$ (whatever the choice of $c_i$) and is the total number of papers in $D$ with the $a_j$ activity.
- thus each confusion matrix $M_{ij}$ is such that $U + V + X + Y = |D|$ where $|D|$ is _the total number of paper_ under scrutiny.


In [ ]:
dataset = pd.read_csv(DATASET_FILENAME, index_col=[0, 1, 2], header=[0, 1, 2])
# dataset.index.names = ["class", "name", "with"]
# dataset.columns.names = ["class", "name", "with"]

all_compounds = set(dataset.index.get_level_values(1))
all_activities = set(dataset.columns.get_level_values(1))

dataset

We can extract the "old" 53\*33 matrix we use in the version version of this software, where we had only the with/with queries.
In our small running example, that would be:

|                   |            | germination | cytotoxicity |
| ----------------- | ---------- | ----------- | ------------ |
|                   |            | allelopathy | pharmaco     |
| alkaloid          | acridine   | 11          | 1296         |
| terpenoid/terpene | triterpene | 32          | 2115         |


In [ ]:
with_with_matrix = dataset.xs("w/", level=2).xs("w/", level=2, axis=1)
with_with_matrix

We do some sanity checks explained earlier :

- the name (level 2 of rows/columns) are _unique_
- the sum of each confusion submatrixes is _constant_ and is the total number of papers $|D|$, here 243 964.


In [ ]:
# sanity check #1
assert dataset.shape == (2 * len(all_compounds), 2 * len(all_activities))
dataset


# sanity check : group by summing on level 2 on both rows and cols produce a matrix of constants : the number of papers
submatrix_sum = dataset.groupby(level=1).sum().groupby(level=1, axis=1).sum()
number_of_papers = np.unique(submatrix_sum.values)
# if the Scopus collection did not evolve during while querying
assert len(number_of_papers) == 1

number_of_papers = number_of_papers[0]
print(f"The domain contains {number_of_papers} papers")

with_with_total = with_with_matrix.values.sum()
print(
    f"Total number of positive/positive occurrences is {with_with_total} for {number_of_papers} papers (average={with_with_total/number_of_papers})"
)


Lets illustrate the content of this table. The **2 by 2 confusion submatrix** about _acridine_ and _cytotoxicity_ is as follows.


In [ ]:
acridine_cytotoxicity_submatrix = dataset.loc[
    (
        "alkaloid",
        "acridine",
    ),
    (
        "pharmaco",
        "cytotoxicity",
    ),
]
print(f"Among {number_of_papers} papers, there are")
for i, j in product(bex.SELECTORS, bex.SELECTORS):
    print(f"{acridine_cytotoxicity_submatrix.loc[i,j]} papers {i} acridine and {j} cytotoxicity in their keywords")

print("The acridine and cytotoxicity confusion matrix is as follows")
acridine_cytotoxicity_submatrix


We compute marginal sums on rows and cols and add them to the orginial dataset.

**NOTE** we actually should do the other way around, that is to query Scopue for margin and for all (w/, w/) cells and _then_ fill in the missing ones.

In [ ]:
margin_idx = (bex.CLASS_SYMB, bex.MARGIN_SYMB, bex.SELECTORS[1])
margin_cols = dataset.groupby(level=1).sum().drop_duplicates().reset_index(drop=True)
margin_cols.index = pd.MultiIndex.from_tuples([margin_idx])

margin_rows = dataset.groupby(level=1, axis=1).sum().iloc[:, 0]
margin_rows.name = margin_idx
margin_rows = pd.DataFrame(margin_rows)

dataset_margins = dataset.copy()
dataset_margins[margin_idx] = margin_rows
dataset_margins = pd.concat([dataset_margins, margin_cols]).fillna(number_of_papers).astype(int)

#TODO add som styling to margins
dataset_margins


In [ ]:
# see https://en.wikipedia.org/wiki/Confusion_matrix
# see https://en.wikipedia.org/wiki/Phi_coefficient



def fowlkes_mallows(arr):
    arr = arr.reshape(2, 2)
    return np.sqrt(arr[1][1] / (arr[1][1] + arr[0][1]) * arr[1][1] / (arr[1][1] + arr[1][0]))


def acc(arr):
    arr = arr.reshape(2, 2)
    return (arr[1][1] + arr[0][0]) / (arr.sum())


def x_score(arr):
    arr = arr.reshape(2, 2)
    return (arr[1][1] + arr[0][0] - arr[0][1] - arr[1][0]) / (arr.sum())

def original(arr):
    arr = arr.reshape(2, 2)
    return arr[1][1]  / (arr.sum())


print(fowlkes_mallows(acridine_cytotoxicity_submatrix.values))
print(acc(acridine_cytotoxicity_submatrix.values))
print(x_score(acridine_cytotoxicity_submatrix.values))
print(original(acridine_cytotoxicity_submatrix.values))
acridine_cytotoxicity_submatrix


In [ ]:
# redimension the values to a 4D array
C, A = len(all_compounds), len(all_activities)
print(C, A)
M = dataset.values.reshape((C, 2, A, 2))
M = np.moveaxis(M, 1, -2)

# M.sum(axis=(2,3)) or similarly
np.sum(M, axis=(2, 3), keepdims=False)



We obtain the same as submatrix_sum


In [ ]:
# 1D is easier for apply_along_axis
M2 = M.reshape((C * A, 4))

print(M2[0])
print(M[0, 0])
print(f"{acc(M[0, 0]) = }")
print(f"{x_score(M[0, 0]) = }")
print(f"{fowlkes_mallows(M[0, 0]) = }")
print("--")

confused = {}
for fnct in [original, acc, x_score, fowlkes_mallows]:
    confused[fnct.__name__] = np.apply_along_axis(fnct, 1, M2).reshape((C, A))

    print(
        f"{fnct.__name__}:\n min={confused[fnct.__name__].min():.3f} mean={confused[fnct.__name__].mean():.3f} max={confused[fnct.__name__].max():.3f} std={confused[fnct.__name__].std():.3f}"
    )
# np.apply_over_axes(f, M2, axes=1)
# confused


In [ ]:
# TODO : les couleurs par classes

ca = prince.CA(n_components=2, n_iter=5, copy=True, check_input=True, engine="auto", random_state=42, benzecri=False)

confused_df = pd.DataFrame(100 * confused)
confused_df.index = with_with_matrix.index  #  set([(x,y) for x,y,_ in dataset.index.to_list()])
confused_df.columns = with_with_matrix.columns
# confused_df.values = confused
# confused_df


/!\ HERE /!\

for (df, name) in confused: # [(with_with_matrix, "Original"), (confused_df, "Confused")]:
    print(f"-------{name}-------")
    ca = ca.fit(df)

    # ca.row_coordinates(df)[:10]
    # ca.column_coordinates(df)[:10]

    pprint(ca.explained_inertia_)
    pprint(ca.col_masses_[:10])
    pprint(ca.eigenvalues_)
    pprint(ca.total_inertia_)

    ax = ca.plot_coordinates(
        X=confused_df,
        ax=None,
        figsize=(12, 12),
        x_component=0,
        y_component=1,
        show_row_labels=True,
        show_col_labels=True,
    )
    plt.show()
